In [1]:
import json

from essentia.streaming import (
    VectorInput,
    FrameCutter,
    TempoCNN,
    TensorflowPredictTempoCNN,
    TensorflowInputTempoCNN,
    TensorflowInputMusiCNN,
    VectorRealToTensor,
    TensorToPool,
    TensorflowPredict,
    PoolToTensor,
    TensorToVectorReal
)
from essentia import Pool, run, reset
from IPython import display
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import softmax
import soundcard as sc

%matplotlib nbagg

2022-01-18 15:40:37.064673: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-18 15:40:37.064702: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-01-18 15:40:37.833146: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-01-18 15:40:37.833306: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-01-18 15:40:37.833317:

Define the analysis parameters.
To make this demo work in real-time, we tweaked some of the analysis parameters of `MusiCNN`.
While it was trained on patches of size 187 (\~3 seconds) we set `patch_size` to 64 (\~1 second) to increase the prediction rate.
You can experiment with the `patch_size` and `display_size` parameters to modify the prediction rate to your taste.

In [2]:
with open('deeptemp-k16-3.json', 'r') as json_file:
    metadata = json.load(json_file)

model_file = 'deeptemp-k16-3.pb'
input_layer = metadata['schema']['inputs'][0]['name']
output_layer = metadata['schema']['outputs'][0]['name']

# Analysis parameters.
sample_rate = 11025
frame_size = 1024 
hop_size = 256
n_bands = 40
patch_size = 64
display_size = 10

buffer_size = patch_size * hop_size

Instantiate the algorithms. With this, we create a network similar to the one used inside `TensorflowPredictMusiCNN`, the wrapper algorithm presented in the previous tutorial. However, by instantiating the algorithms separately we gain additional control required for real-time usage.

In [8]:
buffer = np.zeros(buffer_size, dtype='float32')
vimp = VectorInput(buffer)
fc = FrameCutter(frameSize=frame_size, hopSize=hop_size)
tim = TensorflowInputTempoCNN()
vtt = VectorRealToTensor(shape=[1, 1, patch_size, n_bands],
                         lastPatchMode='discard')
ttp = TensorToPool(namespace=input_layer)
tfp = TensorflowPredict(graphFilename=model_file,
                        inputs=[input_layer],
                        outputs=[output_layer])
tfpt = TensorflowPredictTempoCNN(graphFilename=model_file,
                        input=input_layer,
                        output=output_layer)
ptt = PoolToTensor(namespace=output_layer)
ttv = TensorToVectorReal()
pool = Pool()

[   INFO   ] Successfully loaded graph file: `deeptemp-k16-3.pb`
[   INFO   ] Successfully loaded graph file: `deeptemp-k16-3.pb`


Connect the algorithms. We also store the mel-spectrograms in the `Pool` for visualization purposes.

In [9]:
# vimp.data   >> fc.signal
# fc.frame    >> tim.frame
# tim.bands   >> vtt.frame
# vtt.tensor  >> ttp.tensor
# ttp.pool    >> tfp.poolIn
# tfp.poolOut >> ptt.pool
# ptt.tensor  >> ttv.tensor
# ttv.frame   >> (pool, output_layer)

vimp.data           >>  tfpt.signal
tfpt.predictions    >>  (pool, output_layer)

Create a callback function that will be called every time the audio buffer is ready to process.

Initialize the plots and start processing the loopback stream.

In [10]:
def callback_console(data):
    buffer[:] = data.flatten()

    # Generate predictions.
    reset(vimp)
    run(vimp)
    
    print(softmax(20 * pool[output_layer][-1, :].T))



In [11]:
pool.clear()

# Capture and process the speakers loopback.
with sc.all_microphones(include_loopback=True)[0].recorder(samplerate=sample_rate) as mic:
    while True:
        # callback(mic.record(numframes=buffer_size).mean(axis=1))
        callback_console(mic.record(numframes=buffer_size).mean(axis=1))

KeyError: "no key found named 'output'"